# Importing

Importing the necessary libraries

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns

from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# Data processing

Importing the datasets into a Dataframe

In [4]:
jester_items = pd.read_csv(r'C:\Users\vbraun\Downloads\SDM-Datasets\jester_items.csv')
jester_ratings = pd.read_csv(r'C:\Users\vbraun\Downloads\SDM-Datasets\jester_ratings.csv')
display(jester_items.head(3),jester_ratings.head(3))

,jokeId,jokeText
0,1,"A man visits the doctor. The doctor says ""I ha..."
1,2,This couple had an excellent relationship goin...
2,3,Q. What's 200 feet long and has 4 teeth? \n\nA...


,userId,jokeId,rating
0,1,5,0.219
1,1,7,-9.281
2,1,8,-9.281


As is shown above, only the 'jester_ratings' dataset is required for the creation of our RecSys model.

The jokeId will be renamed to itemId to have a more universal dataset.

In [5]:
jester_df = jester_ratings.rename(columns={'jokeId':'itemId'})
print('Original length of dataset:',len(jester_df))

# Data filtering

In order to filter the dataset based on activity and reduce the sparsity of the data, the data will be grouped and filtered based on jokes and users. The datasets will show how many ratings each joke has gotten and the amount of ratings a user has given.

In [6]:
jester_item_count = jester_df[['itemId','userId']].groupby('itemId').count().reset_index().rename(columns={'userId':'user_count'})
jester_user_count = jester_df[['itemId','userId']].groupby('userId').count().reset_index().rename(columns={'itemId':'item_count'})

display(jester_item_count.head(3),jester_user_count.head(3))

,itemId,user_count
0,5,661
1,7,59122
2,8,57720


,userId,item_count
0,1,62
1,2,34
2,3,18


To reduce the sparcity of data in the dataset, we will filter out users that have rated fewer than 5% of the total amount of jokes.

In [9]:
filtered_jester_df = jester_df[jester_df['userId'].isin(jester_user_count[jester_user_count['item_count']/len(jester_item_count) > 0.05]['userId'])]

print('Length of dataset:',len(filtered_jester_df))

Length of dataset: 1729140


Finally, jokes that have been rated by fewer than 20 people will be filtered out of the dataset.

In [11]:
filtered_jester_df = filtered_jester_df[filtered_jester_df['itemId'].isin(jester_item_count[jester_item_count['user_count']>20]['itemId'])]

print('Length of dataset:',len(filtered_jester_df))

Length of dataset: 1729140
